In [1]:
import os
os.environ['HF_HOME'] = '/run/cache/'

In [2]:
import pandas as pd
# import uuid
pd.set_option('display.max_colwidth', None)
# def add_id_to_data(df):
#     df['id'] = [str(uuid.uuid4())[:10] for _ in range(len(df))]
#     return df
df = pd.read_json('weaviate_input.json')

df.head()

context  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               a few nights ago i talked to this girl i know about my self esteem issues for the first time we talked for hours and she told me time and again that i was a great guy she told me i was attractive and have a great personality etc i really started to feel better about myself by the time i woke up the next morning\nnow though i cant stop thinking about her but i leave to go back to college in a few days and i go to school 4 hours away from her so now i feel constantly depressed because even if i told her how i felt it wouldnt matter i feel helpless and i dont know what to do   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                a few nights ago i talked to this girl i know about my self esteem issues for the first time we talked for hours and she told me time and again that i was a great guy she told me i was attractive and have a great personality etc i really started to feel better about myself by the time i woke up the next morning now though i cant stop thinking about her but i leave to go back to college in a few days and i go to school 4 hours away from her so now i feel constantly depressed because even if i told her how i felt it wouldnt matter i feel helpless and i dont know what to do   
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  a few years ago i was making love to my wife when for no known reason i lost my erection \r\n\r\nnow im in my early 30s and my problem has become more and more frequent  this is causing major problems for my ego and its diminishing my self esteem this has resulted in ongoing depression and tearing apart my marriage \r\n\r\ni am devastated and cannot find a cause for these issues i am very attracted to my wife and want to express it in the bedroom like i used to \r\n\r\nwhat could be causing this and what can i do about it   
3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [3]:
df.isna().sum()

context     0
response    0
document    0
dtype: int64

In [5]:
print(f"Number of unique contexts: {df['context'].nunique()}")

Number of unique contexts: 995


In [7]:
df['response_length'] = df['response'].str.len()
print(df['response_length'].describe())

count      995.000000
mean      2539.940704
std       3524.853501
min          1.000000
25%        739.000000
50%       1367.000000
75%       2839.000000
max      36220.000000
Name: response_length, dtype: float64


In [8]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   35G   66G  35% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   35G   66G  35% /run
tmpfs            14G     0   14G   0% /dev/shm
/dev/nvme2n1    2.0G   16M  1.9G   1% /home/jovyan
tmpfs            14G  124K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G  4.9M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [9]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# Load your data
df1 = df.copy()

# Deduplicate contexts and aggregate responses
df_unique = df.groupby('context')['response'].agg(list).reset_index()

# Load the model and tokenizer
model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def generate_questions(context, num_questions=2):
    input_text = f"Generate {num_questions} questions based on this context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).input_ids
    
    outputs = model.generate(input_ids, max_length=64, num_return_sequences=num_questions, num_beams=4)
    
    questions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return questions

# Generate questions for each unique context
df_unique['generated_questions'] = df_unique['context'].apply(generate_questions)

# Explode the questions to create separate rows
df_ground_truth = df_unique.explode('generated_questions').reset_index(drop=True)

# Rename columns for clarity
df_ground_truth = df_ground_truth.rename(columns={
    'context': 'original_context',
    'generated_questions': 'question'
})

# # Save the ground truth dataset
# df_ground_truth.to_json('data/ground_truth_dataset.json', orient='records', lines=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
import sys

# ... (your existing code to create df_ground_truth)

# Check shape
print(f"Shape of df_ground_truth: {df_ground_truth.shape}")

# Check memory usage
memory_usage = df_ground_truth.memory_usage(deep=True).sum()
print(f"Memory usage of df_ground_truth: {memory_usage / 1e6:.2f} MB")

# Estimate file size (for JSON format)
estimated_size = sys.getsizeof(df_ground_truth.to_json()) / 1e6
print(f"Estimated file size (JSON): {estimated_size:.2f} MB")

# # If the size is acceptable, save the DataFrame
# if estimated_size < 100:  # For example, if it's less than 100 MB
#     df_ground_truth.to_json('data/ground_truth_dataset.json', orient='records', lines=True)
#     print("Data saved successfully.")
# else:
#     print("Data size is too large. Consider reducing the dataset or using a different storage method.")

Shape of df_ground_truth: (1990, 3)
Memory usage of df_ground_truth: 1.18 MB
Estimated file size (JSON): 5.99 MB


In [18]:
df.columns

Index(['context', 'response', 'document', 'response_length'], dtype='object')

In [12]:
df_ground_truth.columns

Index(['original_context', 'response', 'question'], dtype='object')

In [13]:
# Save the questions data
df_ground_truth.to_parquet('ground_truth_data.parquet')

In [15]:
df1 = pd.read_parquet('ground_truth_data.parquet')
df1.shape

(1990, 3)

In [8]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def generate_questions(context, num_questions=2, max_context_length=300):
    # Truncate the context if it's too long
    context_tokens = tokenizer.encode(context, add_special_tokens=False, truncation=True, max_length=max_context_length)
    truncated_context = tokenizer.decode(context_tokens)

    prompt = f"""
    You are someone experiencing mental health challenges. Based on the following context, create {num_questions} questions that you might ask a mental health professional or support group:

    Context: {truncated_context}

    Generate {num_questions} questions:
    """
    
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).input_ids
    
    outputs = model.generate(
        input_ids,
        max_length=150,
        num_return_sequences=num_questions,
        num_beams=num_questions,
        do_sample=True,
        temperature=0.7,
        no_repeat_ngram_size=2
    )
    
    questions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return questions

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
sample_df = df.sample(n=10, random_state=42)

# List to store new rows
new_rows = []

# Generate questions and create new format
for index, row in sample_df.iterrows():
    questions = generate_questions(row['Context'])
    
    for i, question in enumerate(questions):
        new_rows.append({
            'question': question,
#             'documents': i,
            'document': row['document']
        })

# Create new dataframe with the desired format
questions_df = pd.DataFrame(new_rows)

# Reorder columns
questions_df = questions_df[['question', 'document']]

In [10]:
questions_df.head()

,question,document
0,What may be the reason for you being in a wome...,e739b2db-1
1,What may be the reason for you not seeing your...,e739b2db-1
2,How long has it been since your mother's death?,2510448d-8
3,How long has it been since your mother died?,2510448d-8
4,What may be the reason for you to be angry wit...,db959012-6


In [11]:
df.head()

,Context,Response,document
0,a few years ago i was making love to my wife w...,[first step always is to do a medical rule out...,cb02ebd1-4
1,a lot of times i avoid situations where i am t...,[hello and thank you for your question first i...,0c9816ff-e
2,a year ago the love of my life left me and nev...,[who takes care of your son is a significant p...,cd3e9d25-8
3,about 3 years ago or so i was skinny but i was...,[hey i am so impressed with your efforts to as...,42f38c04-1
4,about 5 months ago my ex left without fully ex...,[since you realize your sense of trust was bro...,1141a8ca-3


In [14]:
# sample_df = df.sample(n=10, random_state=42)

# List to store new rows
new_rows = []

# Generate questions and create new format
for index, row in df.iterrows():
    questions = generate_questions(row['Context'])
    
    for i, question in enumerate(questions):
        new_rows.append({
            'question': question,
#             'documents': i,
            'document': row['document']
        })

# Create new dataframe with the desired format
questions_df = pd.DataFrame(new_rows)

# Reorder columns
questions_df = questions_df[['question', 'document']]

In [15]:
questions_df.shape

(1662, 2)

In [17]:
questions_df.sample(10)

,question,document
584,What might I do if I am worried about something?,2616ddbe-a
455,What may be the reason for you wearing lingerie?,64b6dbcf-b
1419,What may be the reason for you to be apart fro...,8a099260-d
1107,What may be the reason for you feeling like this?,02d20d54-6
1299,What may be the reason why you aren't interest...,c0d80d3d-2
1052,What may be the reason you distanced yourself ...,1c309951-0
1557,What might be the reason for you being anxious...,06215297-8
702,What might be the reason you want to move out?,215a6cf0-c
855,What may be the reason she is angry?,0a6fa060-f
118,What may be the reason for you being down and ...,7af158e0-8


In [18]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   38G   63G  38% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   38G   63G  38% /run
tmpfs            14G  4.0K   14G   1% /dev/shm
/dev/nvme2n1    2.0G  3.3M  1.9G   1% /home/jovyan
tmpfs            14G  124K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G  8.2M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [19]:
# Save the original data with IDs
df.to_parquet('mental_health_data_with_ids.parquet')

# Save the questions data
questions_df.to_parquet('ground_truth_data.parquet')

In [20]:
# Load the Parquet file into a DataFrame
df1 = pd.read_parquet('mental_health_data_with_ids.parquet')
df1.shape

(831, 3)

In [22]:
df1.sample(10)

,Context,Response,document
264,im dealing with an illness that will never go ...,[is it possible that even though your illness ...,8c434cba-5
249,im a teenager and i just got my first job i am...,[getting your first job is an exciting terrify...,4ab46a4d-d
634,i think adult life is making him depressed and...,[how do you help yourself to believe you requi...,8b309f9b-2
454,we live with my expartners sister and her husb...,[hi new york im happy to hear that your daught...,aa4f24f2-1
157,i have no friends no hobbies and no interest i...,[hello and thank you for your question it cert...,9a92800e-2
738,my dad beat and mentally abused me so badly th...,[i am so sorry to hear that you are struggling...,d5e21679-8
341,my boyfriend is in recovery from drug addictio...,[my empathy goes out to you relationships are ...,5b00fb0e-2
636,i told him i liked him he called me lovely and...,[well as disappointed as you may very well fee...,87979059-7
564,i got engaged and everything was going well th...,[somehow you knew the marriage wouldnt have a ...,07a4a324-2
78,i been having anger problems a lot lately it o...,[with me knowing that a healthy relationship u...,5b1082aa-e


In [24]:
df2 = pd.read_parquet('ground_truth_data.parquet')
df2.shape

(1662, 2)

In [26]:
df2.tail(10)

,question,document
1652,What might be the reason for you not cutting?,a61f42d3-d
1653,What might be the reason for you not cutting a...,a61f42d3-d
1654,What might you do if you want to cut yourself?,3a65764a-e
1655,What may happen after you cut yourself?,3a65764a-e
1656,What might happen if you move in with your gir...,33b14d46-a
1657,What might happen if you moved in with your gi...,33b14d46-a
1658,What might be the reason for your boyfriend's ...,b5df98a6-c
1659,What might be the reason for your boyfriend's ...,b5df98a6-c
1660,What may be the reason you are attracted to ol...,310dbd5b-8
1661,What may be the reason for your attraction to ...,310dbd5b-8
